In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
## Load dataset
N = 60
data_path = data_dir + "/gmm/gmm_%d" % N
Data = []
# for i in range(10):
data = torch.from_numpy(np.load(data_path + '/obs_%d.npy' % N)).float()
Data.append(data)
## Hyper Parameters
MCMC_STEPS = 10
## Train Parameters
NUM_EPOCHS = 500
K = 3 
D = 2
SAMPLE_SIZE = 10
BATCH_SIZE = 20
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')
PATH = 'apg-v2-smaller-%dpts-%dsamples' % (N, SAMPLE_SIZE)
Train_Params = (NUM_EPOCHS, K, D, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH) 
## Model Parameters
NUM_HIDDEN_GLOBAL = 8
NUM_HIDDEN_LOCAL = 32
LEARNING_RATE =  1e-4

In [3]:
from local_enc import *
from global_oneshot_v2 import *
from global_enc_v2 import *
import torch

def Init_models(K, D, num_hidden_global, num_hidden_local, CUDA, device, lr, RESTORE=False, PATH=None):
    # initialization
    os_eta = Oneshot_eta(K, D, num_hidden_global, CUDA, device)

    f_z = Enc_z(K, D, num_hidden_local, CUDA, device)
    f_eta = Enc_eta(K, D, num_hidden_global, CUDA, device)

    b_z = Enc_z(K, D, num_hidden_local, CUDA, device)
    b_eta = Enc_eta(K, D, num_hidden_global, CUDA, device)

    if CUDA:
        with torch.cuda.device(device):
            os_eta.cuda()
            f_z.cuda()
            f_eta.cuda()
            b_z.cuda()
            b_eta.cuda()
    if RESTORE:
        os_eta.load_state_dict(torch.load("../weights/os-eta-%s" % PATH))
        f_z.load_state_dict(torch.load("../weights/f-z-%s" % PATH))
        f_eta.load_state_dict(torch.load("../weights/f-eta-%s" % PATH))
        b_z.load_state_dict(torch.load("../weights/b-z-%s" % PATH))
        b_eta.load_state_dict(torch.load("../weights/b-eta-%s" % PATH))
    # optimizer =  torch.optim.Adam(list(os_eta.parameters())+list(f_z.parameters())+list(f_eta.parameters()),lr=lr, betas=(0.9, 0.99))
    optimizer =  torch.optim.Adam(list(os_eta.parameters())+list(f_z.parameters())+list(f_eta.parameters())+list(b_z.parameters())+list(b_eta.parameters()),lr=lr, betas=(0.9, 0.99))
    return (os_eta, f_z, f_eta, b_z, b_eta), optimizer

def Save_models(models, path):
    (os_eta, f_z, f_eta, b_z, b_eta) = models
    torch.save(os_eta.state_dict(), "../weights/os-eta-%s" % path)
    torch.save(f_z.state_dict(), "../weights/f-z-%s" % path)
    torch.save(f_eta.state_dict(), "../weights/f-eta-%s" % path)
    
    torch.save(b_z.state_dict(), "../weights/b-z-%s" % path)

In [4]:
models, optimizer = Init_models(K, D, NUM_HIDDEN_GLOBAL, NUM_HIDDEN_LOCAL, CUDA, DEVICE, LEARNING_RATE, RESTORE=False, PATH=None)

In [ ]:
from apg import APG
train(models, APG, optimizer, Data, MCMC_STEPS, Train_Params)

epoch=0\500, data order=tensor([0])


In [ ]:
Save_models(models, PATH)